In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_elections'

In [4]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text)

In [6]:
soup_df = pd.read_html(str(soup.find('table', attrs = {'class': 'wikitable', 'class': 'sortable'})))[0].drop(50)

In [7]:
soup_df.columns = ['state', 'seats', 'dem_seats', 'dem_change', 'rep_seats', 'rep_change']

In [8]:
state_dist = soup_df.drop(['dem_seats', 'dem_change', 'rep_seats', 'rep_change'], axis = 1)

In [9]:
# need list of state abbreviations

abb_url = 'https://www.yourdictionary.com/articles/state-abbreviations'

abb_res = requests.get(abb_url)

abb_soup = BeautifulSoup(abb_res.text)

In [10]:
state_abb = pd.read_html(str(abb_soup.find('table')), header = 0)[0].drop('Traditional Abbreviation', axis = 1)

In [11]:
print(state_dist.columns)
print(state_abb.columns)

Index(['state', 'seats'], dtype='object')
Index(['State Name', 'USPS Abbreviation'], dtype='object')


In [12]:
# join state_dist to state_abb

st_rep = pd.merge(state_dist, state_abb, left_on = 'state', right_on = 'State Name', how = 'inner')\
    .drop('State Name', axis = 1).rename({'USPS Abbreviation': 'state_abbr'}, axis = 1)

st_rep.head(1)

,state,seats,state_abbr
0,Alabama,7,AL


In [13]:
# example url
# https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N.

url_base = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='
url_suffix = '&spec=N'

url_list = []

for r in range(len(st_rep)):
    abbr = st_rep.loc[r]['state_abbr']
    rep_num = st_rep.loc[r]['seats']

    for d in range(1, rep_num + 1):
        url = f"{url_base}{abbr}{str(d).zfill(2)}{url_suffix}"
        url_list.append(url)

In [17]:
pd.DataFrame(url_list, columns = ['url']).to_csv('../data/dist_urls.csv', index = False)

In [19]:
url_list

['https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL05&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL06&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL07&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AK01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ05&spec=N',
 'https://www.opensecrets